In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
file_path = './assets/Frankenstein.txt'

loader = TextLoader(file_path=file_path, encoding='utf=8').load()

docs = RecursiveCharacterTextSplitter(  # Dividir por capítulo
    separators=[r'Chapter \d+'],
    is_separator_regex=True,
    chunk_overlap=0
    ).split_documents(loader)

for i, doc in enumerate(docs, start=1):
    doc.metadata = {'chapter': i}

In [ ]:
documents = []
for doc in docs:
    chunks = RecursiveCharacterTextSplitter(
        separators=[r'\n\s*\n+'],  # Dividir cada capítulo por parágrafo 
        is_separator_regex=True,
        chunk_overlap=0,
        chunk_size=1000  # Caso não esteja definido, o regex não funciona
 
    ).split_documents([doc])  # Para não dar erro
    for i, chunk in enumerate(chunks, start=1):
        chunk.metadata = {'chapter': doc.metadata['chapter'], 'paragraph': i}
        documents.append(chunk)

In [8]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

c:\Users\Windows11\AppData\Local\pypoetry\Cache\virtualenvs\langchain-llm-9c8IAL59-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
db = Chroma.from_documents(
    documents, embeddings, persist_directory='./assets/chroma_db_Frankenstein'
)  

In [20]:
retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={"k": 5, 'score_threshold': 0.4}
)

In [21]:
prompt = ChatPromptTemplate.from_template("""
Responda a pergunta em português com base no contexto fornecido abaixo.
Inclua o capítulo e o número do parágrafo em sua resposta, para que assim o usuário possa saber a fonte da informação.
                                           
Contexto:                                   
{context}

Entrada do usuário:
{question}
""")


llm = OpenAI(model="gpt-4o-mini")

rag_chain = ( 
    {"context": retriever, "question": RunnableLambda(lambda x: x)}
    | prompt
    | llm
)

response = rag_chain.invoke("What were the main reasons that led Frankenstein's creature to feel isolated and rejected, and how did Victor Frankenstein contribute to this feeling, according to the text?")

In [ ]:
if 'Assistant<|im_sep|>' in response:
    response.replace('Assistant<|im_sep|>', '')  # Artefato da resposta
print(response)

 
Resposta:
As razões principais que levaram a criatura de Frankenstein a se sentir isolada e rejeitada incluem sua aparência horrenda e a reação avassaladora das pessoas que a encontravam, que a viam como um monstro. Além disso, a falta de aceitação e amor por parte de Victor Frankenstein, seu criador, contribuiu significativamente para essa sensação de solidão. Victor, ao invés de assumir a responsabilidade por sua criação e proporcionar apoio, apenas a rejeitou e a levou a um estado de desespero. O texto menciona que, embora a criatura desejasse se conectar com os outros, sua aparência causava medo e repulsa, resultando em sua exclusão da sociedade. 

Essa informação pode ser encontrada no Capítulo 24, parágrafo 21 e no Capítulo 24, parágrafo 44.
